Decorator Pattern
Adds functionality to objects dynamically without modifying their structure.

In [ ]:
def uppercase_decorator(func):
    def wrapper(text):
        return func(text).upper()
    return wrapper

@uppercase_decorator
def greet(name):
    return f"Hello, {name}"

# Example usage:
print(greet("Alice"))  # Output: HELLO, ALICE


## Logging Decorator for CI/CD Pipeline Steps
In DevOps, it’s important to log actions performed in your pipeline, like building, testing, or deploying. The Decorator Pattern allows you to add logging functionality dynamically to any part of your pipeline.

In [1]:
import time

# Step decorator to log the start and end of the action
def log_step(func):
    def wrapper(*args, **kwargs):
        print(f"Starting: {func.__name__}")
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Completed: {func.__name__} in {end_time - start_time:.2f} seconds")
        return result
    return wrapper

# Example pipeline steps with decorators
class Pipeline:
    @log_step
    def build(self):
        print("Building the application...")
        time.sleep(2)

    @log_step
    def test(self):
        print("Running tests...")
        time.sleep(1)

    @log_step
    def deploy(self):
        print("Deploying to production...")
        time.sleep(3)

# Usage
pipeline = Pipeline()
pipeline.build()
pipeline.test()
pipeline.deploy()


Starting: build
Building the application...
Completed: build in 2.00 seconds
Starting: test
Running tests...
Completed: test in 1.00 seconds
Starting: deploy
Deploying to production...
Completed: deploy in 3.00 seconds


## Authentication Decorator for API Endpoints
In DevOps, securing APIs or services is critical. A decorator can be used to add authentication to any API call without modifying the core logic of the endpoint.

In [2]:
# Decorator to check authentication
def require_authentication(func):
    def wrapper(*args, **kwargs):
        if not args[0].authenticated:
            raise PermissionError("Authentication required")
        return func(*args, **kwargs)
    return wrapper

class APIService:
    def __init__(self, authenticated=False):
        self.authenticated = authenticated

    @require_authentication
    def get_data(self):
        return "Sensitive data"

    @require_authentication
    def post_data(self, data):
        print(f"Posting data: {data}")

# Usage
service = APIService(authenticated=True)
print(service.get_data())  # Should succeed

service = APIService(authenticated=False)
try:
    print(service.get_data())  # Should raise PermissionError
except PermissionError as e:
    print(e)


Sensitive data
Authentication required


## Retry Decorator for Deployment Steps
In DevOps, you might want to automatically retry certain steps (like deployment) if they fail. A retry decorator can add this functionality dynamically to any step.

In [11]:
import random

# Decorator to retry a function if it fails
def retry(times=3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(times):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt == times - 1:
                        raise
                    else:
                        print("Retrying...")
        return wrapper
    return decorator

class Deployment:
    @retry(times=3)
    def deploy_to_staging(self):
        print("Deploying to Staging...")
        if random.random() < 0.5:  # Simulate failure
            raise Exception("Deployment failed")
        print("Deployment to Staging successful")

# Usage
deployment = Deployment()
try:
    deployment.deploy_to_staging()  # Retry if it fails
except Exception as e:
    print(f"Final failure: {e}")


Deploying to Staging...
Attempt 1 failed: Deployment failed
Retrying...
Deploying to Staging...
Attempt 2 failed: Deployment failed
Retrying...
Deploying to Staging...
Attempt 3 failed: Deployment failed
Final failure: Deployment failed


## Cache Decorator for Task Results
In DevOps, you might want to cache the results of a task (e.g., a long-running operation like a build) to avoid repeating expensive operations. A cache decorator can be used to achieve this.

In [12]:
# Cache decorator to avoid re-executing long tasks
def cache(func):
    cached_results = {}

    def wrapper(*args, **kwargs):
        key = (args, tuple(kwargs.items()))
        if key not in cached_results:
            print("Cache miss. Executing task...")
            cached_results[key] = func(*args, **kwargs)
        else:
            print("Cache hit. Returning cached result...")
        return cached_results[key]

    return wrapper

class BuildService:
    @cache
    def build_application(self, app_name):
        print(f"Building application {app_name}...")
        time.sleep(2)  # Simulate build process
        return f"Build completed for {app_name}"

# Usage
build_service = BuildService()

# First call will execute the build
print(build_service.build_application("App1"))

# Second call with same arguments will return cached result
print(build_service.build_application("App1"))


Cache miss. Executing task...
Building application App1...
Build completed for App1
Cache hit. Returning cached result...
Build completed for App1
